In [22]:
import pandas as pd
from google.cloud import storage

# Cloud Storage etails
bucket_name = 'customer_compliant'
file_path = 'Consumer_Complaints.csv'

# Local file path to download data
local_file_path = 'Customercomplaints.csv'

# Authenticate to Google Cloud Storage
storage_client = storage.Client()

# Download the file from Cloud Storage
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_file_path)

df = pd.read_csv(local_file_path, low_memory=False)



In [23]:
df.head(4)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/29/2013,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,07/30/2013,Closed with explanation,Yes,No,468882
1,07/29/2013,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,07/31/2013,Closed with explanation,Yes,No,468889
2,07/29/2013,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,07/31/2013,Closed,Yes,No,468879
3,07/29/2013,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,07/30/2013,Closed with explanation,Yes,No,468949


In [24]:
#shape and size of dataset
df.shape, df.size

((670598, 18), 12070764)

In [25]:
df['Sub-issue'].isnull().sum(), df['Consumer complaint narrative'].isnull().sum(), df['Company public response'].isnull().sum(),df['Consumer consent provided?'].isnull().sum()

(400730, 555894, 525401, 462447)

In [26]:
df.isna().sum()

Date received                        0
Product                              0
Sub-product                     198202
Issue                                0
Sub-issue                       400730
Consumer complaint narrative    555894
Company public response         525401
Company                              0
State                             5305
ZIP code                          5324
Tags                            575868
Consumer consent provided?      462447
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?               41419
Complaint ID                         0
dtype: int64

Transfroming Data based on Requirements:

As we see above data had a lot of missing values "NaN", Handing Missing values is key concept in Expoloratory Data Analysis(EDA).

Some common steps are 

1.Replacing Missing Values with most freq values.

2.Droping Rows of Missing Value

In [27]:
df1= df.copy()

In [28]:
df1 = df1.drop(['Sub-issue','Consumer complaint narrative','Company public response','Consumer consent provided?'], axis=1)


In [29]:
df1= df1.drop(['Tags'], axis=1)

In [30]:
df1 = df1.drop(['Sub-product'],axis=1)

In [31]:
df1 = df1.drop(['Complaint ID'], axis=1)

In [32]:
df1['Product'].value_counts()

Mortgage                   210324
Debt collection            124236
Credit reporting           119195
Credit card                 79007
Bank account or service     76084
Consumer Loan               26604
Student loan                21651
Payday loan                  4786
Money transfers              4720
Prepaid card                 3171
Other financial service       806
Virtual currency               14
Name: Product, dtype: int64

In [33]:
df1['Company'].value_counts().count()

3933

In [34]:
df1['Submitted via'].value_counts()

Web            446035
Referral       123224
Phone           47389
Postal mail     43869
Fax              9733
Email             348
Name: Submitted via, dtype: int64

In [35]:
df['Company'].value_counts()

Bank of America                       61720
Wells Fargo & Company                 48902
Equifax                               41771
Experian                              39473
JPMorgan Chase & Co.                  38928
                                      ...  
Southern Funding Alliance USA, LLC        1
R.C. Temme Corporation                    1
DeGrasse & Rolnick                        1
Wetherington Hamilton, P.A.               1
Cross State Motors, LLC                   1
Name: Company, Length: 3933, dtype: int64

In [36]:
df1['Consumer disputed?'].value_counts()

No     496466
Yes    132713
Name: Consumer disputed?, dtype: int64

In [37]:
df1 = df1.dropna(subset='Consumer disputed?')
df1 = df1.dropna(subset=['ZIP code'])

In [38]:
df1.isnull().sum()

Date received                   0
Product                         0
Issue                           0
Company                         0
State                           1
ZIP code                        0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
dtype: int64

In [39]:
df1['State']= df1['State'].fillna('CA')


Here, We created a column shows Response time by the company, Before that we converted the object >> Datetime and find the number of days it takes to replay recived answer

In [40]:
df1['Date sent to company'] = pd.to_datetime(df1['Date sent to company'])
df1['Date received'] = pd.to_datetime(df1['Date received'])


In [41]:
df1['Response_time'] = (df1['Date sent to company'] - df1['Date received']).dt.days


After all transfromations done now convert the dataframe to Csv file store in local space.To import Datawarehouse

In [42]:
df1.head(6)

,Date received,Product,Issue,Company,State,ZIP code,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Response_time
0,2013-07-29,Consumer Loan,Managing the loan or lease,Wells Fargo & Company,VA,24540,Phone,2013-07-30,Closed with explanation,Yes,No,1
1,2013-07-29,Bank account or service,Using a debit or ATM card,Wells Fargo & Company,CA,95992,Web,2013-07-31,Closed with explanation,Yes,No,2
2,2013-07-29,Bank account or service,"Account opening, closing, or management",Santander Bank US,NY,10065,Fax,2013-07-31,Closed,Yes,No,2
3,2013-07-29,Bank account or service,Deposits and withdrawals,Wells Fargo & Company,GA,30084,Web,2013-07-30,Closed with explanation,Yes,No,1
4,2013-07-29,Mortgage,"Loan servicing, payments, escrow account",Franklin Credit Management,CT,06106,Web,2013-07-30,Closed with explanation,Yes,No,1
5,2013-07-29,Bank account or service,Deposits and withdrawals,Bank of America,TX,75025,Web,2013-07-30,Closed with explanation,Yes,No,1


In [45]:
df1.to_csv("Consumer_Complaints_transformed.csv")

In [46]:
processed_file_path = 'Consumer_Complaints_transformed.csv'
blob = bucket.blob(processed_file_path)
blob.upload_from_filename(processed_file_path)

In [1]:
import pandas as pd
from google.cloud import storage

# Cloud Storage etails
bucket_name = 'customer_compliant'
file_path = 'Consumer_Complaints_transformed.csv'

# Local file path to download data
local_file_path = 'Consumer_Complaints_transformed.csv'

# Authenticate to Google Cloud Storage
storage_client = storage.Client()

# Download the file from Cloud Storage
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_file_path)

df = pd.read_csv(local_file_path, low_memory=False)

In [4]:
df1 = df.copy()

In [12]:
df1 = df1.drop(['ZIP code'],axis=1)

In [14]:
df1.to_csv("Consumer_Complaints_transform.csv")

In [15]:
processed_file_path = 'Consumer_Complaints_transform.csv'
blob = bucket.blob(processed_file_path)
blob.upload_from_filename(processed_file_path)